In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully.")

In [ ]:
# Load the mock backtest results
# This file would be the real output from your Backtrader run
RESULTS_FILE = "../data/backtest_results_log.csv"

try:
    results_df = pd.read_csv(RESULTS_FILE, index_col='date', parse_dates=True)
    print("Backtest results loaded successfully!")
    results_df.head()
except FileNotFoundError as e:
    print(f"Error loading data: {e}")
    print("Please ensure you have run the `create_mock_data.py` script first.")

In [ ]:
# --- Plot the Equity Curve ---
# This is the most fundamental backtest visualization.
plt.figure(figsize=(14, 7))
plt.plot(results_df['equity'], label='Portfolio Equity', color='blue')
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend()
plt.show()

In [ ]:
# --- Calculate Key Performance Indicators (KPIs) ---
# We'll write functions to calculate common metrics. In a real project,
# these would live in a dedicated module in `src/`.

def calculate_sharpe_ratio(returns, periods_per_year=252):
    """Calculates the annualized Sharpe ratio."""
    # Assuming risk-free rate is 0
    return np.sqrt(periods_per_year) * (returns.mean() / returns.std())

def calculate_max_drawdown(equity_curve):
    """Calculates the maximum drawdown."""
    high_water_mark = equity_curve.cummax()
    drawdown = (equity_curve - high_water_mark) / high_water_mark
    return drawdown.min()

In [ ]:
# Calculate and print the KPIs
returns = results_df['equity'].pct_change().dropna()

total_return = (results_df['equity'][-1] / results_df['equity'][0]) - 1
sharpe_ratio = calculate_sharpe_ratio(returns)
max_dd = calculate_max_drawdown(results_df['equity'])
cagr = ((results_df['equity'][-1] / results_df['equity'][0]) ** (1 / (len(results_df)/252))) - 1


print("--- Backtest Performance Metrics ---")
print(f"Total Return: {total_return:.2%}")
print(f"Compound Annual Growth Rate (CAGR): {cagr:.2%}")
print(f"Annualized Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Maximum Drawdown: {max_dd:.2%}")

In [ ]:
# --- Plot the Drawdown Curve ---
# Visualizing drawdowns helps understand the risk and pain points of a strategy.
high_water_mark = results_df['equity'].cummax()
drawdown = (results_df['equity'] - high_water_mark) / high_water_mark

plt.figure(figsize=(14, 7))
plt.fill_between(drawdown.index, drawdown * 100, 0, color='red', alpha=0.3)
plt.plot(drawdown.index, drawdown * 100, color='red', lw=1)
plt.title('Drawdown Curve')
plt.xlabel('Date')
plt.ylabel('Drawdown (%)')
plt.show()